In [1]:
import xgboost as xgb
import pandas as pd

In [2]:
# 読み出し
data = pd.read_pickle('data.pkl')
nomination_onehot = pd.read_pickle('nomination_onehot.pkl')
selected_performers_onehot = pd.read_pickle('selected_performers_onehot.pkl')
selected_directors_onehot = pd.read_pickle('selected_directors_onehot.pkl')
selected_studio_onehot = pd.read_pickle('selected_studio_onehot.pkl')
selected_scriptwriter_onehot = pd.read_pickle('selected_scriptwriter_onehot.pkl')
review_dataframe = pd.read_pickle('review_dataframe.pkl')
tfidf = pd.read_pickle('tfidf.pkl')

In [20]:
table = pd.concat([
    data[['prize', 'title', 'year', 'screen_time']],
    nomination_onehot,
    selected_performers_onehot,
    selected_directors_onehot,
    selected_studio_onehot,
    selected_scriptwriter_onehot
], axis = 1)

In [ ]:
for year in range(1978, 2019 + 1):
    rg = xgb.XGBRegressor(silent= True)
    X = table.query('year != {}'.format(year)).drop(['prize', 'title', 'year'], axis = 1).values
    y = table.query('year != {}'.format(year))['prize'].values
    rg.fit(X,y)
    result = rg.predict(table.query('year == {}'.format(year)).drop(['prize', 'title', 'year'], axis = 1).values)
    prize = table.query('year == {}'.format(year))
    title = table.query('year == {}'.format(year))['title'].copy()
    title[prize['prize'] == 1] = title[prize['prize'] == 1].map(lambda s: '★' + s)
    print(year)
    print(pd.Series(result, index = title.values).sort_values(ascending=False) )
    print('')

In [3]:
frames = [
    data.query('year == 2004')[['title', 'production_studio', 'other_nominates']],
    review_dataframe
         ]

def asdf(s):
    s['len'] = len(s['reviews'])
    return s
    
pd.concat(
    frames,
    axis = 1,
    join = 'inner'
).apply(asdf, axis = 1).drop(['reviews'], axis = 1)

,title,production_studio,other_nominates,len
id,,,,
131,壬生義士伝,[松竹],[],32
132,阿修羅のごとく,[東宝],[nikkan_sports],16
133,踊る大捜査線 THE MOVIE 2 レインボーブリッジを封鎖せよ！,[東宝],[golden_gross],55
134,座頭市,"[松竹, オフィス北野]",[],55
135,スパイ・ゾルゲ,[東宝],[],3


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=20)
pca.fit(tfidf.values)
tfidf_df = pd.DataFrame(pca.transform(tfidf.values), index = tfidf.index)

In [54]:
table = pd.concat([
    data[['prize', 'title', 'year']],
    tfidf
], axis = 1)

In [ ]:
for year in range(1978, 2019 + 1):
    rg = xgb.XGBRegressor(silent= True)
    X = table.query('year != {}'.format(year)).drop(['prize', 'title', 'year'], axis = 1).values
    y = table.query('year != {}'.format(year))['prize'].values
    rg.fit(X,y)
    result = rg.predict(table.query('year == {}'.format(year)).drop(['prize', 'title', 'year'], axis = 1).values)
    prize = table.query('year == {}'.format(year))
    title = table.query('year == {}'.format(year))['title'].copy()
    title[prize['prize'] == 1] = title[prize['prize'] == 1].map(lambda s: '★' + s)
    print(year)
    print(pd.Series(result, index = title.values).sort_values(ascending=False) )
    print('')